In [7]:
from bs4 import BeautifulSoup
import urllib.request
import requests
from pprint import pprint
import pandas as pd
import json
import copy
from datetime import datetime, timezone
from google.cloud import storage
import gcsfs
import re
from typing import List

In [8]:
storage_client = storage.Client(project="ytone-430507")
today = datetime.now()
table_name = "douyin_price"
api_name = "get_author_marketing_info"

In [9]:
price_data = []
price_columns = ["video_type", "settlement_type", "is_open", "origin_price", "price", "has_discount", "influencer_id"]

In [10]:
processing_blobs = [
{
    "blob": blob,
    "date": datetime.strptime(blob.name.split('/')[1], "%Y-%m-%d"),
    "batch_number": int(blob.name.split('/')[-1].replace(".json", "").split("_")[-1]),
} for blob in storage_client.list_blobs("0_raw_data",prefix="2_xingtu/") if api_name in blob.name]
processing_blobs

[{'blob': <Blob: 0_raw_data, 2_xingtu/2024-09-11/get_author_marketing_info_0911_0.json, 1726037773893626>,
  'date': datetime.datetime(2024, 9, 11, 0, 0),
  'batch_number': 0},
 {'blob': <Blob: 0_raw_data, 2_xingtu/2024-09-17/get_author_marketing_info_0917_0.json, 1726565276671612>,
  'date': datetime.datetime(2024, 9, 17, 0, 0),
  'batch_number': 0},
 {'blob': <Blob: 0_raw_data, 2_xingtu/2024-09-18/get_author_marketing_info_0918_0.json, 1726633855345648>,
  'date': datetime.datetime(2024, 9, 18, 0, 0),
  'batch_number': 0},
 {'blob': <Blob: 0_raw_data, 2_xingtu/2024-09-19/get_author_marketing_info_0919_0.json, 1726719807493916>,
  'date': datetime.datetime(2024, 9, 19, 0, 0),
  'batch_number': 0},
 {'blob': <Blob: 0_raw_data, 2_xingtu/2024-10-09/get_author_marketing_info_241009_0.json, 1728451162976669>,
  'date': datetime.datetime(2024, 10, 9, 0, 0),
  'batch_number': 0},
 {'blob': <Blob: 0_raw_data, 2_xingtu/2024-10-11/get_author_marketing_info_1011_0.json, 1728632678124038>,
  'dat

In [11]:
processed_blobs = [{
    "blob": blob,
    "date": datetime.strptime(blob.name.split('/')[2], "%Y-%m-%d"),
    "batch_number": int(blob.name.split('/')[-1].replace(".parquet", "").split("_")[-1]),
} for blob in storage_client.list_blobs("4_data_warehouse",prefix="1_xingtu/") if table_name == blob.name.split("/")[1] and len(blob.name.split("/")) == 4]
processed_blobs

[{'blob': <Blob: 4_data_warehouse, 1_xingtu/douyin_price/2024-09-08/douyin_price_240924_0.parquet, 1727152779136509>,
  'date': datetime.datetime(2024, 9, 8, 0, 0),
  'batch_number': 0},
 {'blob': <Blob: 4_data_warehouse, 1_xingtu/douyin_price/2024-09-08/douyin_price_240924_1.parquet, 1727152986107737>,
  'date': datetime.datetime(2024, 9, 8, 0, 0),
  'batch_number': 1},
 {'blob': <Blob: 4_data_warehouse, 1_xingtu/douyin_price/2024-09-11/douyin_price_240924_0.parquet, 1727153351296496>,
  'date': datetime.datetime(2024, 9, 11, 0, 0),
  'batch_number': 0},
 {'blob': <Blob: 4_data_warehouse, 1_xingtu/douyin_price/2024-09-17/douyin_price_240924_0.parquet, 1727154021846218>,
  'date': datetime.datetime(2024, 9, 17, 0, 0),
  'batch_number': 0},
 {'blob': <Blob: 4_data_warehouse, 1_xingtu/douyin_price/2024-09-18/douyin_price_240924_0.parquet, 1727154023696727>,
  'date': datetime.datetime(2024, 9, 18, 0, 0),
  'batch_number': 0},
 {'blob': <Blob: 4_data_warehouse, 1_xingtu/douyin_price/2024-

In [12]:
to_process = []
for processing_blob in processing_blobs:
    if processing_blob["date"] >= datetime(2024, 9, 13):
        if processing_blob["batch_number"] not in [processed_blob["batch_number"] for processed_blob in processed_blobs if processing_blob["date"] == processed_blob["date"]]:
            to_process.append(processing_blob)
pprint(to_process)
print(len(to_process))

[{'batch_number': 0,
  'blob': <Blob: 0_raw_data, 2_xingtu/2024-10-11/get_author_marketing_info_1011_0.json, 1728632678124038>,
  'date': datetime.datetime(2024, 10, 11, 0, 0)}]
1


In [16]:
def get_price_author_marketing_info_data(raw_data):
    result = []
    for data in raw_data:
        get_author_marketing_info = data.get("get_author_marketing_info")
        profile_id = data["profile_id"]
        if get_author_marketing_info:
            price_info = get_author_marketing_info.get("price_info")
            if price_info:
                for price in price_info:
                    record = {}
                    record["video_type"] = price["video_type"]
                    record["settlement_type"] = price["settlement_type"]
                    record["is_open"] = price["is_open"]
                    if "origin_price" in price:
                        record["origin_price"] = price["origin_price"]
                    if "price" in price:
                        record["price"] = price["price"]
                    record["has_discount"] = price["has_discount"]
                    record["influencer_id"] = profile_id
                    result.append(record)
    return result


In [17]:
for item in to_process:
    raw_data = json.loads(item["blob"].download_as_string())
    price_data = get_price_author_marketing_info_data(raw_data)
    df = pd.DataFrame(price_data, columns=price_columns)
    df.to_parquet("gs://4_data_warehouse/1_xingtu/douyin_price/" + str(item["date"].date()) + "/douyin_price_" + today.strftime("%y%m%d") + "_" + str(item["batch_number"]) + ".parquet")